# CAFEPP (utilising cafepp.py)

***

## This example will produce a set of monthly SST files across the V1 assimilation run (interpolated onto a 1x1 latxlon grid), and upon success, generate an average over the nino3.4 region and plot the time-series.

***

# Various settings required:

## BATCH determines whether it will be sent to the queue via qsub command or run interactively.

## CLEAN determines whether the run directory will be emptied prior to processing.

Choose to run either 1. assimilation 2. forecast, then continue with plotting options.

In [8]:
print('BEGIN')

from __future__ import print_function #this is to allow print(,file=xxx) feature

import datetime
import shutil
import os
import sys
import socket
import re
import inspect

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

print('sys.version=',sys.version)
hostname=socket.gethostname()
print('hostname=',hostname)

#dvar='tos' #variable to generate as well as further proces and plot.
#dvar='pr'
#dvar='psl'
#dvar='siconc'
#dvar='msftyz' #was called msftyyz
#dvar='ts'
#dvar='thetao'
#dvar='tos'
#dvar='siconc'

BATCH=True #submit to queue
BATCH=False #run interactively but in a batch temporary area.

CLEAN=False #don't remove rundir, just use it.
CLEAN=True #remove rundir and recreate it.

if(BATCH):
  print('Submitting to queue.')

if(CLEAN):
  print('Removing run directory and reestablish it.')

if(re.match('raijin',hostname)):
  cmipdir='/short/v14/mac599' #this might be different to predir for other users.
  predir='/short/v14/mac599' #this is directory area for temporary cafepp files.
elif(re.match('oa-32-cdc',hostname) or re.match('oa-33-cdc',hostname)):
  cmipdir='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6' #this might be different to predir for other users.
  predir='/OSM/CBR/OA_DCFP/data/col414' #this is directory area for temporary cafepp files.
elif(re.match('tube-hba',hostname)):
  cmipdir='/OSM/HBA/OA_DECADAL_CLIMATE/work/col414' #this might be different to predir for other users.
  predir='/OSM/HBA/OA_DECADAL_CLIMATE/work/col414' #this is directory area for temporary cafepp files.
else:
  raise SystemExit('hostname not known:'+__file__+' line number: '+str(inspect.stack()[0][2])) 

topdir=predir+'/'+'cafepp'
#script='cafepp_monthly_assimilation.py'
script='cafepp_monthly_forecast.py'
#script='cafepp_monthly_forecast_v2.py'
#script='cafepp_monthly_control.py'

rundir=topdir+'/'+'rundir20171124143249'
rundir=topdir+'/'+'rundir20171128165302'
rundir=topdir+'/'+'rundir'+datetime.datetime.today().strftime('%Y%m%d%H%M%S')
rundir=topdir+'/'+'rundir' #temporary running directory

rundir=topdir+'/'+'rundir' #temporary running directory
print('Running in directory '+rundir)
print('Using script '+script)

if(re.match('raijin',hostname)):
  srcdir='/home/599/mac599/decadal' #location of main cafepp code
  prodir=srcdir+'/paper_analysis' #project directory
elif(re.match('oa-32-cdc',hostname) or re.match('oa-33-cdc',hostname)):
  srcdir='/OSM/CBR/OA_DCFP/work/col414/cafepp' #'/home/599/mac599/decadal' #location of main cafepp code
  prodir='/OSM/CBR/OA_DCFP/work/col414/cafepp' #srcdir+'/paper_analysis' #project directory
elif(re.match('tube-hba',hostname)):
  srcdir='/OSM/HBA/OA_DECADAL_CLIMATE/work/col414/cafepp' #'/home/599/mac599/decadal' #location of main cafepp code
  prodir='/OSM/HBA/OA_DECADAL_CLIMATE/work/col414/cafepp' #srcdir+'/paper_analysis' #project directory  
print('END')

BEGIN
sys.version= 2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 17:05:42) 
[GCC 7.2.0]
hostname= oa-32-cdc
Removing run directory and reestablish it.
Running in directory /OSM/CBR/OA_DCFP/data/col414/cafepp/rundir
Using script cafepp_monthly_forecast.py
END


# Here we make directories, copy across necessary JSON and python code.

In [ ]:
print('BEGIN')

print('copying json files, generating symlinks, cmor tables, queue script, if necessary.')
if(os.path.exists(rundir) and CLEAN):
  shutil.rmtree(rundir)
  os.mkdir(rundir)
  os.mkdir(rundir+'/'+'JsonTemplates') # exist_ok=True only python3
elif(not os.path.exists(rundir)):
  os.mkdir(rundir)
  os.mkdir(rundir+'/'+'JsonTemplates') # exist_ok=True only python3

if(os.path.isdir(cmipdir+'/'+'CMIP6')):
  print('hello')
  os.symlink(cmipdir+'/'+'CMIP6',rundir+'/'+'CMIP6')
else:
  print('there')
  os.mkdir(cmipdir)
os.symlink(prodir+'/'+'TablesTemplates',rundir+'/'+'TablesTemplates')
os.symlink(prodir+'/'+'cmip6-cmor-tables',rundir+'/'+'cmip6-cmor-tables')

if(BATCH):
  print('Copying and editing '+prodir+'/'+'qjob.csh to '+rundir+'/'+'qjob.csh')
  ifh=open(prodir+'/'+'qjob.csh')
  ofh=open(rundir+'/'+'qjob.csh','w')
  for i,line in enumerate(ifh):
    line=line.replace('CAFEPP_SCRIPT','./'+script+' RUNDIR')
    line=line.replace('RUNDIR',rundir)
    line=line.replace('CONDA_SOURCE','. /short/v14/mac599/anaconda3/etc/profile.d/conda.sh')
    line=line.replace('CONDA_ACTIVATE','conda activate cafepp_27_scipy')
    print(line,file=ofh,end='')
  ifh.close()
  ofh.close()

vector_string=['decadal_diag.py','cafepp.py','app_funcs.py','ProcTime.py']
for i,file_now in enumerate(vector_string):
  print('Copying '+srcdir+'/'+file_now+' to ',rundir)
  shutil.copyfile(srcdir+'/'+file_now,rundir+'/'+file_now)

vector_string=[script]
for i,file_now in enumerate(vector_string):
  print('Copying '+prodir+'/'+file_now+' to ',rundir)
  shutil.copyfile(prodir+'/'+file_now,rundir+'/'+file_now)
  os.chmod(rundir+'/'+script,500) #make executable incase want to run it interacively from terminal.
  
vector_string=[]
#vector_string.append(script) #may need to do edits?
#vector_string.append('cafepp_monthly_assimilation.json')
vector_string.append('JsonTemplates'+'/'+'cafepp_experiments.json')
vector_string.append('JsonTemplates'+'/'+'cafepp_csiro-gfdl.json')
vector_string.append('JsonTemplates'+'/'+'cafepp_vars.json')
#vector_string.append('cafepp_monthly_forecast.py')

for i,file_now in enumerate(vector_string):
  print('Copying '+prodir+'/'+file_now+' to ',rundir)
  shutil.copyfile(prodir+'/'+file_now,rundir+'/'+file_now)
  
print('Copying '+prodir+'/'+'cafepp_monthly_assimilation.json',rundir+'/'+'JsonTemplates/cafepp_monthly_assimilation.json')
shutil.copyfile(prodir+'/'+'cafepp_monthly_assimilation.json',rundir+'/'+'JsonTemplates/cafepp_monthly_assimilation.json')
  
if(not os.path.exists(rundir+'/'+'CMIP5')):
  if(re.match('raijin',hostname)):
    print('No need to set CMIP5 symbolic link on raijin.')
  elif(re.match('oa-32-cdc',hostname) or re.match('oa-33-cdc',hostname)):
    os.symlink('/OSM/CBR/OA_DCFP/data/CAFEPP/g/data/p66/mac599/CMIP5',rundir+'/'+'CMIP5')
  elif(re.match('tube-hba',hostname)):
    os.symlink('/OSM/HBA/OA_DECADAL_CLIMATE/work/col414/CMIP5',rundir+'/'+'CMIP5')
  else:
    raise SystemExit('Dont know how to set CMIP5 symbolic link on this machine.'+__file__+' line number: '+str(inspect.stack()[0][2]))

#   srcdir='/home/599/mac599/decadal' #location of main cafepp code
#   prodir=srcdir+'/paper_analysis' #project directory
 
print('prodir=',prodir)
print('rundir=',rundir)

#break
print('END')

# We establish where to execute the job.

## We import a function that is relatively simple to loop over all necessary years, months, ensembles as required.
## Different applications will require a different module to be written (often small and relatively simple).

In [ ]:
print('BEGIN')

os.chdir(rundir)
print('Current Working Directory=',os.getcwd())

os.environ['APP_OUTPATH'] = '.'
print(os.getenv('APP_OUTPATH'))

import getpass
import numpy as np
import numpy.ma as ma
import os
from time import strftime
import netCDF4
from math import radians, cos, sin, asin, sqrt
import seawater
#import sys
import getopt
import string
from decadal_diag import MustHaveAllLevs,diag_acc_drake,diag_acc_africa,diag_mozmbq,diag_aabw,diag_nadw,\
diag_pp,diag_nflux,diag_ep,diag_ssh,diag_moc,diag_moc_atlantic,diag_moc_pacific,diag_moc_indian,\
diag_shice_cover,diag_nhice_cover,diag_nino34,xtra_nino34,init_data,sum_data,avg_data,filemonth_index,\
data_wavg,time_avg,diag_nhblocking_index,diag_rws5,finish,diag_msftyz,make_mask3D,diag_mfo,transPort,\
diag_rws500,create_odirs,create_ofils,diag_iod,diag_iod,xtra_iod,atmos_vertical_interpolate,diag_isothetaoNc,\
calc_iso_surface,calc_isoN,grab_var_meta,diag_psl,diag_hfls,diag_heat_content,diag_salt_content,\
diag_north_heat_trans,diag_north_salt_trans,ocean_vertical_interpolate,diag_thetao0to80m,diag_varNl,\
uncomment_json,process_json,modify_json,get_daily_indices_for_monthlyave,diag_maxdTbydz,diag_depmaxdTbydz,\
diag_dTbydz,shade_2d_simple,shade_2d_latlon,diag_zmld_boyer,zmld_boyer,sigmatheta,diag_zmld_so,\
zmld_so,diag_spice,spice,diag_bigthetao,diag_soabs,diag_spiciness,diag_potrho,fractional_year_from_num2date,\
new_monthly_array_shape,restrict_input_files,get_timestamp_number
from decadal_diag import data_wavg_ProcTime,cmor_file_parts,cmor_directory_parts,cmor_ripf_parts,file_spec_summary,generate_daily_month_indices
from decadal_diag import plot_map_box,plot_xy_climatology,plot_xy_ensemble,calculate_monthly_climatology_anomaly_from_monthly

import cmor
import cdtime
from app_funcs import *
import json
import pprint
from datetime import date
import filecmp
from shutil import copyfile
import cdms2
import inspect
import socket
import glob
from matplotlib.mlab import griddata
import scipy.sparse as sps
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point
import matplotlib as mpl
mpl.rcParams['mathtext.default'] = 'regular'
import matplotlib.pyplot as plt
from gridfill import fill as poisson_fill

#from ProcTime import ProcTime

#del(cafepp_monthly_forecast)
#if('cafepp_monthly_forecast' not in sys.modules):
#  print('no')
#else:
#  print('yes')

if(script=='cafepp_monthly_assimilation'):
  import cafepp_monthly_assimilation #this script/function needs to be edited to limit variable.
elif(script=='cafepp_monthly_forecast.py'):
  import cafepp_monthly_forecast #this script/function needs to be edited to limit year,month,ensemble range,variable if desired. Will need to restart kernel for it to take effect.
elif(script=='cafepp_monthly_control.py'):
  import cafepp_monthly_control

#print(sys.modules)

print('END')

## information common to processing all runs...

In [4]:
print('BEGIN.')

import math

#raise Exception('STOP!')

#del(get_timestamp_number)
#from decadal_diag import get_timestamp_number

nmy=12

rad = 4.0*math.atan(1.0)/180.0
#print('rad=',rad)

lons_nino = [[190, 240, 240, 190, 190], [210, 270, 270, 210, 210], [160, 210, 210, 160, 160], [270, 280, 280, 270, 270]] # indices_nino=['nino34','nino3','nino4','nino1+2']
lats_nino = [[-5, -5, 5, 5, -5], [-5.1, -5.1, 5.1, 5.1, -5.1], [-5, -5, 5, 5, -5], [-10, -10, 0, 0, -10]] # 170W-120W:190-240, 150W-90W:210-270, 160E-150W:160-210, 90W-80W:270-280 (10S-0 latitude, others all 5S to 5N)
indices_nino=['nino34','nino3','nino4','nino1+2'] # 170W-120W:190-240, 150W-90W:210-270, 160E-150W:160-210, 90W-80W:270-280 (10S-0 latitude, others all 5S to 5N)
indices_label=['Ni$\~{n}$o3.4','Ni$\~{n}$o3','Ni$\~{n}$o4','Ni$\~{n}$o1+2']
  
#if(cafe_grid=='gr2'):
#elif(cafe_grid=='gn'):

indices_i_gr2,indices_j_gr2=[[190,239],[210,269],[150,209],[270,280]],[[85,94],[85,94],[85,94],[80,89]] #check this, whether I need +1 also...what about fractional cells?
indices_i_gn,indices_j_gn=[[110,159],[130,189],[80,129],[190,199]],[[122,151],[122,151],[122,151],[107,136]] #check this, whether I need +1 also...what about fractional cells?
indices_i_ncep2,indices_j_ncep2=[[102,128],[112,144],[86,112],[144,149]], [[45,50],[45,50],[45,50],[42,46]]

#indices_select=array('i',[0,1,2,3])
#indices_select=[np.int(0)]
#indices_select=range(3)
#print(type(indices_select))
#print('indices_select=',indices_select)

#used for speeding things up, either can choose a single indice or all - don't know why can't use : in eval for e.g.
sss='-1' #last
sss='0' #first
sss='ALL'

#raise Exception('STOP!')

if(sss!='ALL'):
  indices_nino=[indices_nino[eval(sss)]]
  indices_label=[indices_label[eval(sss)]]
  indices_i_gr2=[indices_i_gr2[eval(sss)]]
  indices_j_gr2=[indices_j_gr2[eval(sss)]]
  indices_i_gn=[indices_i_gn[eval(sss)]]
  indices_j_gn=[indices_j_gn[eval(sss)]]
  indices_i_ncep2=[indices_i_ncep2[eval(sss)]]
  indices_j_ncep2=[indices_j_ncep2[eval(sss)]]
print('indices_nino=',indices_nino)

print('indices_i_gr2=',indices_i_gr2)

nindices_nino=len(indices_nino)

#else:
#  raise Exception('STOP!')
#raise Exception('STOP!')
#del(get_timestamp_number)
#from decadal_diag import get_timestamp_number

rad = 4.0*math.atan(1.0)/180.0
#print('rad=',rad)

print('END.')

BEGIN.
indices_nino= ['nino34', 'nino3', 'nino4', 'nino1+2']
indices_i_gr2= [[190, 239], [210, 269], [150, 209], [270, 280]]
END.


# 1. Here we are procesing an assimilation, set script='cafepp_monthly_assimilation.py' above.

## Either submit it to the queue or run interactively.

In [5]:
print('BEGIN')

SWITCH_OFF=True #temporary for easy skipping of this section.

if(not SWITCH_OFF):
  print('prodir=',prodir)

  if(script=='cafepp_monthly_assimilation'):
    if(BATCH):
      os.chmod(script,500)
      os.system('qsub '+rundir+'/'+'qjob.csh')
    else:
      print('Current Working Directory=',os.getcwd())
      os.chdir(rundir)
      cafepp_monthly_assimilation.main(rundir)
  
print('END')

BEGIN
END



# 2. Here we are processing multi-ensemble forecast, set script='cafepp_monthly_forecast.py' above.
## Either submit it to the queue or run interactively.

queued version needs more work...


In [7]:
print('BEGIN')

#from importlib import reload
#import importlib
#print(dir(importlib))
#cafepp_monthly_forecast=importlib.import_module(cafepp_monthly_forecast)
#raise Exception('STOP!')

SWITCH_OFF=True #temporary for easy skipping of this section.
SWITCH_OFF=False

if(not SWITCH_OFF):
  #print('prodir=',prodir)
  
  dvar='tos'
  dvar='msftyz'

  cafe_experiment='v1_forecast'
  ybeg=2003
  yend=2003
  mbeg=1
  mend=12
  ybeg_first=2002
  yend_last=2016
  mbeg_first=2
  mend_last=5
  ebeg=1
  eend=1

  line_kwargs='rundir='+rundir+' dvar='+dvar+' cafe_experiment='+cafe_experiment+ \
    ' ybeg='+str(ybeg)+' yend='+str(yend)+' mbeg='+str(mbeg)+' mend='+str(mend)+ \
    ' ybeg_first='+str(ybeg_first)+' yend_last='+str(yend_last)+ \
    ' mbeg_first='+str(mbeg_first)+' mend_last='+str(mend_last)+ \
    ' ebeg='+str(ebeg)+' eend='+str(eend)  
  
  if(script=='cafepp_monthly_forecast.py'):

    if(BATCH):
      os.chmod(script,500)
      
      print('Copying and editing '+prodir+'/'+'qjob.csh to '+rundir+'/'+'qjob.csh')
      ifh=open(prodir+'/'+'qjob.csh')
      ofh=open(rundir+'/'+'qjob.csh','w')
      for i,line in enumerate(ifh):
        line=line.replace('CAFEPP_SCRIPT','./'+script+' '+line_kwargs)
        line=line.replace('RUNDIR',rundir)
        line=line.replace('CONDA_SOURCE','. /short/v14/mac599/anaconda3/etc/profile.d/conda.sh')
        line=line.replace('CONDA_ACTIVATE','conda activate cafepp_27_scipy')
        print(line,file=ofh,end='')
      ifh.close()
      ofh.close()
      
      os.system('qsub '+rundir+'/'+'qjob.csh')
    else:
      print('Current Working Directory=',os.getcwd())
      os.chdir(rundir)
      
      kwargs=dict(x.split('=', 1) for x in line_kwargs.split(' '))
      cafepp_monthly_forecast.main(**kwargs)
  
#      cafepp_monthly_forecast.main(rundir, dvar=dvar, cafe_experiment='v1_forecast', \
#        ybeg=2002, yend=2016, mbeg=1, mend=12, \
#        ybeg_first=2002, yend_last=2016, mbeg_first=2, mend_last=5, \
#        ebeg=1, eend=11)
    
#      cafepp_monthly_forecast.main(rundir, dvar=dvar, cafe_experiment=cafe_experiment, \
#        ybeg=2015, yend=2015, mbeg=1, mend=1, \
#        ybeg_first=2002, yend_last=2016, mbeg_first=2, mend_last=5, \
#        ebeg=1, eend=11)  
  
print('END')

BEGIN
Current Working Directory= /OSM/CBR/OA_DCFP/data/col414/cafepp/rundir
cafe_experiment,dvar,ybeg,yend,mbeg,mend,ybeg_first,yend_last,mbeg_first,mend_last,ebeg,eend= v1_forecast msftyz 2003 2003 1 12 2002 2016 2 5 1 1
Processing cafepp_monthly_assimilation.json
top_directory_no2= /g/data1/v14/forecast/v1/yr2003/mn1/OUTPUT.1
Processing cafepp_experiments.json
MAIN
hostname= oa-32-cdc
Running cafepp from JSON instructions: cafepp.json
fh_printfile= <ipykernel.iostream.OutStream object at 0x7f0fa3ee4a90>
cafepp_experiments_fh= {
"v0":{
"source_id":                    "CAFE-1-0",
"source":                       "CAFE-1-0",
"experiment_id":"piControl",
"experiment":"pre-industrial control",
"parent_experiment_id":"piControl-spinup",
"history":"Coupled (ocean-atmosphere-land-ice-seaice) CAFE experiment using GFDL coupled model source code",
"confluence_notes":"Original version of the coupled model using the ACCESS ocean grid with the GFDL namelist options from the CM2p1 set-up of their c

json_input_var_fh= {
"defaults":{
"info":"None", 
"area_t":"None",
"area_u":"None"
},
"siconc":{
"info":"None",
"area_t":"True",
"inputs":"CN",
"realm":"ice",
"ounits":"%",
"table":"SImon,SIday",
"table_frequency":"month,daily",
"ovars":"dvar",
"OutputVarStructure":"time_lat_lon",
"output_type":"diagnostic",
"plev_type":"None",
"lat_lon_type":"y_T,x_T",
"aaa_newlevs_type":"int",
"interp_fill_options":"griddata_scipy",
"diagnostic_args_string":"100.0,fh_printfile",
"diagnostic_function_name":"siconc"
},
"tos":{
"info":"None",
"area_t":"True",
"inputs":"sst",
"inputs_alternative":"temp", 
"realm":"ocean", 
"ounits":"degC", 
"table":"Omon,Oday", 
"table_frequency":"month,daily", 
"ovars":"dvar", 
"OutputVarStructure":"time_lat_lon", 
"positive":"None", 
"output_type":"broadcast", 
"plev_type":"None", 
"lat_lon_type":"y_T,x_T", 
"aaa_newlevs_type":"int", 
"interp_fill_options":"griddata_scipy"
},
"sos":{
"info":"None",
"area_t":"True",
"inputs":"salt",
"realm":"ocean",
"ounits":"0.001",
"t

['msftyz']
('Number of original/new input files=', 6, 6)
('self.input_files=', ['/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2003/mn1/OUTPUT.1/ocean_month_2003_01.nc', '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2003/mn1/OUTPUT.1/ocean_month_2004_01.nc', '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2003/mn1/OUTPUT.1/ocean_month_2005_01.nc', '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2003/mn1/OUTPUT.1/ocean_month_2006_01.nc', '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2003/mn1/OUTPUT.1/ocean_month_2007_01.nc', '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2003/mn1/OUTPUT.1/ocean_month_2008_01.nc'])
('Found season definition', 'MON', ' in self.season_map with indices ', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ' file:ProcTime line number: 253')
('self.time.shape=', (72,))
('self.time=', <class 'netCDF4._netCDF4._Variable'>
float64 time('time',)
    long_name: time
    units: days since 0001-01-01 00:00:00
    cartesian

data_wavg_Proctime:data.shape= (12, 50, 300, 360)
data_wavg_Proctime:levels= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
data_wavg_ProcTime: ivar= ty_trans_gm
data_wavg_Proctime:ProcTimenpvalues= [ 0  1  2  3  4  5  6  7  8  9 10 11]
data_wavg_Proctime:ProcTimenpvalues2= [ 0  1  2  3  4  5  6  7  8  9 10 11]
data_wavg_Proctime:data.shape= (12, 50, 300, 360)
data_wavg_Proctime:levels= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
data.shape= (12, 3, 50, 300)
ntimes_passed= 12
ProcTimennpvalues= 12
ProcTimeN.time_bounds= [[ 731230.  731261.]
 [ 731261.  731289.]
 [ 731289.  731320.]
 [ 731320.  731350.]
 [ 731350.  731381.]
 [ 731381.  731411.]
 [ 731411.  731442.]
 [ 731442.  731473.]
 [ 731473.  731

data_wavg_ProcTime: ivar= ty_trans_gm
data_wavg_Proctime:ProcTimenpvalues= [ 0  1  2  3  4  5  6  7  8  9 10 11]
data_wavg_Proctime:ProcTimenpvalues2= [36 37 38 39 40 41 42 43 44 45 46 47]
data_wavg_Proctime:data.shape= (12, 50, 300, 360)
data_wavg_Proctime:levels= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
data.shape= (12, 3, 50, 300)
ntimes_passed= 12
ProcTimennpvalues= 12
ProcTimeN.time_bounds= [[ 731230.  731261.]
 [ 731261.  731289.]
 [ 731289.  731320.]
 [ 731320.  731350.]
 [ 731350.  731381.]
 [ 731381.  731411.]
 [ 731411.  731442.]
 [ 731442.  731473.]
 [ 731473.  731503.]
 [ 731503.  731534.]
 [ 731534.  731564.]
 [ 731564.  731595.]
 [ 731595.  731626.]
 [ 731626.  731655.]
 [ 731655.  731686.]
 [ 731686.  731716.]
 [ 731716.  731747.]
 [ 731747.  731777.]
 [ 731777.  731808.]
 [ 731808.  731839.]
 [ 731839.  731869.]
 [ 731869.

SystemExit: Forced exit file:cafepp_monthly_forecast.py line number: 143

In [ ]:
print('BEGIN.')

SWITCH_OFF=True #temporary for easy skipping of this section.
#SWITCH_OFF=False

if(not SWITCH_OFF):
  
  import math
  os.chdir(rundir)

  colors=['black','red','green','blue','orange','brown','purple','pink']
  
  #forc_nens=11
  grid_label='gr2'
  grid_label='gn'
  forc_files_stringA='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-9]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-9]i1p1f1_'+grid_label+'_2011??-??????.nc'
  forc_files_stringB='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-9]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-9]i1p1f1_'+grid_label+'_2012??-??????.nc'
  forc_files_stringC='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-9]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-9]i1p1f1_'+grid_label+'_2013??-??????.nc'
  forc_files_stringD='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-9]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-9]i1p1f1_'+grid_label+'_2014??-??????.nc'

  forc_files_stringA=''
  forc_files_stringD=''

  forc_files=sorted(restrict_input_files(glob.glob(forc_files_stringA)+glob.glob(forc_files_stringB)+glob.glob(forc_files_stringC)+glob.glob(forc_files_stringD),24,72)) #later want to split these into different ensembles. Will assume same input files for all valid ensembles.

  nforc_files=len(forc_files)
  print('forc_files[0]=',forc_files[0])
  #print('forc input_files ('+str(nforc_files)+') =',forc_files)

  datetime_all,datetime_uniq,ripf_all,ripf_uniq=file_spec_summary(forc_files,True)

  #raise Exception('STOP!')
print('END.')

# 3. Here we are processing uninitialised control experiment, set script='cafepp_monthly_control.py' above

Either submit it to the queue or run interactive.

# turn off scrolling on output cells...

In [ ]:
#%%javascript
#IPython.OutputArea.prototype._should_scroll = function(lines) {
#    return false;
#}

In [ ]:
print('BEGIN')


# %load_ext autoreload
# %autoreload 2

%matplotlib inline

SWITCH_OFF=True #temporary for easy skipping of this section.
SWITCH_OFF=False

cafe_experiment='v0'
#cafe_experiment='v1'
#cafe_experiment='v2'
#cafe_experiment='v3'

#cafe_experiment='v0' #t_surf in atmos_month_*nc files.
ybeg=491
yend=500
mbeg=1
mend=12

#dvar='so'
#dvar='thetao'
#dvar='tos'
#dvar='tas'
#dvar='siconc'
dvar='msftyz' #was called msftyyz

NoClobber=True #do not clobber
NoClobber=False #clobber

line_kwargs='NoClobber='+str(NoClobber)+' rundir='+rundir+' dvar='+dvar+' cafe_experiment='+cafe_experiment+ \
  ' ybeg='+str(ybeg)+' yend='+str(yend)+' mbeg='+str(mbeg)+' mend='+str(mend)

if(not SWITCH_OFF):
  
  print('prodir=',prodir)
  
  if(script=='cafepp_monthly_control.py'):
    if(BATCH):
      os.chmod(script,500)
    
      print('Copying and editing '+prodir+'/'+'qjob.csh to '+rundir+'/'+'qjob.csh')
      ifh=open(prodir+'/'+'qjob.csh')
      ofh=open(rundir+'/'+'qjob.csh','w')
      for i,line in enumerate(ifh):
        line=line.replace('CAFEPP_SCRIPT','./'+script+' '+line_kwargs)
        line=line.replace('RUNDIR',rundir)
        line=line.replace('CONDA_SOURCE','. /short/v14/mac599/anaconda3/etc/profile.d/conda.sh')
        line=line.replace('CONDA_ACTIVATE','conda activate cafepp_27_scipy')
        print(line,file=ofh,end='')
      ifh.close()
      ofh.close()
  
      os.system('qsub '+rundir+'/'+'qjob.csh')
    else:
      print('Current Working Directory=',os.getcwd())
      os.chdir(rundir)

      kwargs=dict(x.split('=', 1) for x in line_kwargs.split(' '))
      cafepp_monthly_control.main(**kwargs)
  
print('END')

In [ ]:

raise Exception('STOP!')

# Process/Plot control run

In [ ]:
print('BEGIN.')

pval=5
#pval=1

run='historial'
run='piControl'

cont_files=['CMIP6/CMIP/CSIRO/CAFE-1-0/'+run+'/r1i1p'+str(pval)+'f1/Omon/tos/gn/v20171025/tos_Omon_'+run+'_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_009101-010012.nc', \
            'CMIP6/CMIP/CSIRO/CAFE-1-0/'+run+'/r1i1p'+str(pval)+'f1/Omon/tos/gn/v20171025/tos_Omon_'+run+'_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_019101-020012.nc', \
            'CMIP6/CMIP/CSIRO/CAFE-1-0/'+run+'/r1i1p'+str(pval)+'f1/Omon/tos/gn/v20171025/tos_Omon_'+run+'_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_029101-030012.nc', \
            'CMIP6/CMIP/CSIRO/CAFE-1-0/'+run+'/r1i1p'+str(pval)+'f1/Omon/tos/gn/v20171025/tos_Omon_'+run+'_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_039101-040012.nc', \
            'CMIP6/CMIP/CSIRO/CAFE-1-0/'+run+'/r1i1p'+str(pval)+'f1/Omon/tos/gn/v20171025/tos_Omon_'+run+'_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_049101-050012.nc']

#cont_files=cont_files[0:2]

cont_nexps=len(cont_files)

for cont_cnt,cont_file in enumerate(cont_files):
  
  print('cont input file=',cont_file)

  cont_ifh=netCDF4.Dataset(cont_file)

  cont_lat=cont_ifh.variables['latitude'][:,0]
  cont_lon=cont_ifh.variables['longitude'][0,:]

  #forc_clat=np.zeros((nforc_files),dtype=float) #if latitude range changes will need to have custom clat.
  cont_clat=np.cos(cont_lat[:]*rad)

  cont_time=cont_ifh.variables['time']

  cont_date_time_stamp=netCDF4.num2date(cont_time[:],cont_time.units,cont_time.calendar)

  cont_ntime=len(cont_time)

  '''
  print('date_time_stamp=',date_time_stamp)
  num_stamp=netCDF4.date2num(date_time_stamp,time.units,time.calendar)
  print('num_stamp=',num_stamp)
  print('year_fraction=',year_fraction)
  '''
  
  if(cont_cnt==0):
    cont_year_fraction_monthly=ma.zeros((cont_ntime,cont_nexps),dtype=float)
    cont_nino_monthly=ma.zeros((cont_ntime,nindices_nino,cont_nexps),dtype=float)    
  cont_year_fraction_monthly[:,cont_cnt]=fractional_year_from_num2date(cont_date_time_stamp,cont_time.calendar)

  #print('type(cont_date_time_stamp)=',type(cont_date_time_stamp))
  #print('cont_date_time_stamp=',cont_date_time_stamp)

  #raise Exception('STOP!')
  for cont_k,indice in enumerate(indices_nino):
    #cont_k=indices_nino.index(indice)
    cont_imin,cont_imax=indices_i_gn[indices_nino.index(indice)][0],indices_i_gn[indices_nino.index(indice)][1]
    cont_jmin,cont_jmax=indices_j_gn[indices_nino.index(indice)][0],indices_j_gn[indices_nino.index(indice)][1]
    #print('indices.index(indice),cont_imin,cont_imax,cont_jmin,cont_jmax=',indices.index(indice),cont_imin,cont_imax,cont_jmin,cont_jmax)
    #print('cont_lat,cont_lon=',cont_lat[cont_jmin:cont_jmax],cont_lon[cont_imin:cont_imax])

    cont_nino_monthly[:,cont_k,cont_cnt]=np.average(np.average(cont_ifh.variables[dvar][:,cont_jmin:cont_jmax,cont_imin:cont_imax],axis=1,weights=cont_clat[cont_jmin:cont_jmax]),axis=1)
  cont_ifh.close()
  #indices_i_gn,indices_j_gn=[[122,152],[122,152],[122,152]],[[110,160],[110,160],[110,160]] #check this, whether I need +1 also...
  #raise Exception('STOP!')
  #print('cont_year_fraction=',cont_year_fraction_monthly)

  %matplotlib inline

  fix,ax=plt.subplots()
  for cont_k,indice in enumerate(indices_nino):
    ax.plot(cont_year_fraction_monthly[:,cont_cnt],cont_nino_monthly[:,indices_nino.index(indice),cont_cnt],label=indices_label[indices_nino.index(indices_nino[cont_k])])
  legend=ax.legend(loc='lower right',shadow=False,fontsize='x-large')
  plt.title('Monthly values (CONTROL run)')
  plt.xlabel('Year')
  plt.ylabel('$^o$C')
  #plt.show()

#raise Exception('STOP!')

  cont_ybeg=cont_date_time_stamp[0].year
  cont_yend=cont_date_time_stamp[-1].year

  cont_ydiff=cont_yend-cont_ybeg+1
  nmy=12

  cont_MissingMonths=False

  cont_first_month=cont_date_time_stamp[0].month
  cont_last_month=cont_date_time_stamp[-1].month

  cont_missing_months_beg,cont_missing_months_end=0,0

  cont_cyear_beg_skip,cont_cyear_end_skip=0,1

  if(cont_first_month!=1):
    cont_missing_months_beg=12-cont_first_month
    cont_cyear_beg_skip=1
    cont_MissingMonths=True

  if(cont_last_month!=12):
    cont_missing_months_end=12-cont_last_month
    cont_cyear_end_skip=2
    cont_MissingMonths=True
  
  if(cont_MissingMonths):
    print('There are missing months in the set. '+str(cont_missing_months_beg)+' at beginning and '+str(cont_missing_months_end)+' at end.')
    print('Currently years with missing months are not used in generating long term monthly climatology.')
    print('And missing months will be set to missing in the final time-series.')
    cont_ts_beg,cont_ts_end,cont_ts_avg,cont_dt_beg,cont_dt_end,cont_dt_avg=get_timestamp_number(cont_ybeg,cont_yend,1,12,cont_time.units,cont_time.calendar)
    cont_year_fraction_monthly_full=cont_fractional_year_from_num2date(cont_ts_avg,cont_time.calendar)
  
    cont_nino_monthly_full=ma.masked_all(nindices_nino,cont_ydiff*nmy,dtype=float) #ensure missing months are masked out.
    cont_last_month_index=cont_ydiff*nmy-cont_last_month
    cont_nino_monthly_full[cont_first_month-1:cont_last_month_index,:]=cont_nino_monthly
  else:
    print('All years have 12 months.')
  
    cont_nino_monthly_full=cont_nino_monthly
    cont_year_fraction_monthly_full=cont_year_fraction_monthly
    
#print('cont_nino_monthly_full.shape=',cont_nino_monthly_full.shape)

cont_nino_climatology,cont_nino_monthly_anomaly = calculate_monthly_climatology_anomaly_from_monthly(cont_nino_monthly_full)

%matplotlib inline

for plot_this_exp,cont_file in enumerate(cont_files):
  cont_zero=np.zeros(len(cont_year_fraction_monthly_full[:,plot_this_exp]))
  fix,ax=plt.subplots()
  for cont_k,indice in enumerate(indices_nino):
    ax.plot(cont_year_fraction_monthly_full[:,plot_this_exp],cont_nino_monthly_anomaly[:,cont_k,plot_this_exp],label=indices_label[indices_nino.index(indices_nino[cont_k])])
  plt.plot(cont_year_fraction_monthly_full[:,plot_this_exp],cont_zero,color='black')

  legend=ax.legend(loc='lower right',shadow=False,fontsize='x-large')
  plt.title('Monthly anomalies (CONTROL run)')
  plt.xlabel('Year')
  plt.ylabel('$^o$C')
  plt.show()

print('END.')

# Now to plot data, this will depend on success of previous steps producing necessary outputs with cafepp. Note that we have data on the native grid, where we can discover the points that are in the nino3.4 region.

# Process/Plot forecast run, each forecast is a seperate run.

Find minimum and maximum year/month from the set of input files
Create a dummy time stamp that spans the entire set
Read in data into a multi dimensional array (indice, ensemble, forecast, time)
Have a set of indices which describe the beginning and ending of each forecast, for quick retrieval

Would like to have anomalies relative to various climatologies, these could include different year periods, experiments to form climatology or even zero climatology.

In [ ]:
print('BEGIN.')

import math
os.chdir(rundir)

colors=['black','red','green','blue','orange','brown','purple','pink']
  
#forc_nens=11
grid_label='gr2'
grid_label='gn'
forc_files_stringA='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-3]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-3]i1p1f1_'+grid_label+'_2011??-??????.nc'
forc_files_stringB='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-3]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-3]i1p1f1_'+grid_label+'_2012??-??????.nc'
forc_files_stringC='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-3]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-3]i1p1f1_'+grid_label+'_2013??-??????.nc'
forc_files_stringD='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r[1-3]i1p1f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r[1-3]i1p1f1_'+grid_label+'_2014??-??????.nc'

forc_files_stringA=''
forc_files_stringD=''

forc_files=sorted(restrict_input_files(glob.glob(forc_files_stringA)+glob.glob(forc_files_stringB)+glob.glob(forc_files_stringC)+glob.glob(forc_files_stringD),24,72)) #later want to split these into different ensembles. Will assume same input files for all valid ensembles.

nforc_files=len(forc_files)
print('forc_files[0]=',forc_files[0])
#print('forc input_files ('+str(nforc_files)+') =',forc_files)

datetime_all,datetime_uniq,ripf_all,ripf_uniq=file_spec_summary(forc_files,False)

#print('len(datetime_uniq)=',len(datetime_uniq))
#print('len(datetime_all)=',len(datetime_all))
#print('len(ripf_uniq)=',len(ripf_uniq))
#print('len(ripf_all)=',len(ripf_all))

#print(ripf_all)

#if(len(datetime_all)!=(len(datetime_uniq)*len(ripf_uniq))):
#  print('Possible issue with datetime_uniq,ripf_uniq.')
  
#if(len(datetime_all)!=len(ripf_all)):
#  print('Possible issue with datetime/ripf.')

#raise Exception('STOP!')

'''for forc_i,forc_input_file in enumerate(forc_files):
  print('forc_i,forc_input_file=',forc_i,forc_input_file)

  forc_input_file_tail=os.path.basename(forc_input_file)
  forc_var,forc_table,forc_experiment,forc_model,forc_ripf,forc_grid,forc_datetime=cmor_file_parts(forc_input_file_tail)
  forc_rval,forc_ival,forc_pval,forc_fval=cmor_ripf_parts(forc_ripf)
  forc_input_file_head=string.split(forc_input_file,sep=forc_input_file_tail)[0]
  forc_version,forc_grid,forc_var,forc_table,forc_ripf,forc_experiment,forc_model,forc_institution,forc_activity,forc_cmip=cmor_directory_parts(forc_input_file_head)

  #print('forc_input_file_tail=',forc_input_file_tail)
  #print('forc_input_file_head=',forc_input_file_head)

  #print('forc_var,forc_table,forc_experiment,forc_model,forc_ripf,forc_grid,forc_datetime=',forc_var,forc_table,forc_experiment,forc_model,forc_ripf,forc_grid,forc_datetime)
  
  #raise Exception('STOP!')
  #print('forc_version,forc_grid,forc_var,forc_table,forc_ripf,forc_experiment,forc_model,forc_institution,forc_activity,forc_cmip=',forc_version,forc_grid,forc_var,forc_table,forc_ripf,forc_experiment,forc_model,forc_institution,forc_activity,forc_cmip)
  
  #print('forc_ripf=',forc_ripf)
  #print('forc_rval,forc_ival,forc_pval,forc_fval',forc_rval,forc_ival,forc_pval,forc_fval)
  
  #raise Exception('STOP!')'''
  
#raise Exception('STOP!')

forc_ens=[]
forc_ifhs=[]
forc_date_time_stamps=[]
forc_times=[]
forc_mins=[]
forc_maxs=[]
for forc_i,forc_input_file in enumerate(forc_files):
  #print('forc_i,forc_input_file=',forc_i,forc_input_file)
  forc_input_file_tail=os.path.basename(forc_input_file)
  forc_input_file_head=string.split(forc_input_file,sep=forc_input_file_tail)[0]
  forc_var,forc_table,forc_experiment,forc_model,forc_ripf,forc_grid,forc_datetime=cmor_file_parts(forc_input_file_tail)
  forc_rval,forc_ival,forc_pval,forc_fval=cmor_ripf_parts(forc_ripf)
  forc_version,forc_grid,forc_var,forc_table,forc_ripf,forc_experiment,forc_model,forc_institution,forc_activity,forc_cmip=cmor_directory_parts(forc_input_file_head)
  forc_ens.append(forc_rval)
  #print('forc_i=',forc_i)
  forc_ifhs.append(netCDF4.Dataset(forc_input_file))
  forc_times.append(forc_ifhs[forc_i].variables['time'])
  
  #print(forc_ifhs[forc_i].variables['time'][:])
  
  #raise Exception('STOP!')                 

  if(forc_i>0):  #check that same calendar/units are in use, otherwise would need to convert to common one.
    if(forc_times[forc_i].calendar!=forc_times[forc_i-1].calendar): raise SystemExit('calendars not matching:'+__file__+' line number: '+str(inspect.stack()[0][2]))
    if(forc_times[forc_i].units!=forc_times[forc_i-1].units): raise SystemExit('units not matching:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  forc_mins.append(min(forc_times[forc_i]))
  forc_maxs.append(max(forc_times[forc_i]))
  forc_date_time_stamps.append(netCDF4.num2date(forc_times[forc_i][:],forc_times[forc_i].units,forc_times[forc_i].calendar))
  if(forc_i==0):
    npforc_date_time_stamps=netCDF4.num2date(forc_times[forc_i][:],forc_times[forc_i].units,forc_times[forc_i].calendar)
  else:
    npforc_date_time_stamps=np.append(npforc_date_time_stamps,netCDF4.num2date(forc_times[forc_i][:],forc_times[forc_i].units,forc_times[forc_i].calendar))

np.set_printoptions(threshold='nan') #will print out whole array
np.set_printoptions(threshold=1000)

#print('npforc_date_time_stamps=',npforc_date_time_stamps)
#print('npforc_date_time_stamps.shape=',npforc_date_time_stamps.shape)

#raise Exception('STOP!')                 
#print('type(forc_date_time_stamps)=',type(forc_date_time_stamps))

#cont_date_time_stamp=netCDF4.num2date(cont_time[:],cont_time.units,cont_time.calendar)

#forc_year_fraction_monthly=fractional_year_from_num2date(npforc_date_time_stamps,forc_times[0].calendar) #get working...

#print('forc_times=',forc_times[:])
#print('len(forc_times)=',len(forc_times))

forc_nptimes=np.array(forc_times)
#print('forc_nptimes=',forc_nptimes[:])
#print('type(forc_nptimes)=',type(forc_nptimes))

forc_npftimes=forc_nptimes.flatten()
#print('forc_npftimes=',forc_npftimes[:])

#print(forc_ens)
#print('forc_year_fraction_monthly=',forc_year_fraction_monthly[:])

#raise Exception('STOP!')

year_min=netCDF4.num2date(min(forc_mins),forc_times[0].units,forc_times[0].calendar)
year_max=netCDF4.num2date(max(forc_maxs),forc_times[0].units,forc_times[0].calendar) #not year maximum as need to consider end of each experiment.

print('Forecast start times go from ',min(forc_mins),'to',max(forc_maxs),'or',year_min,'to',year_max)
print('Ensembles go from ',min(forc_ens),'to',max(forc_ens)) #later on restrict to only valid ensembles to keep arrays compact eg 1,2,3,11 but not 1-11.

npforc_ens=np.array(list(set(forc_ens)))
#print(npforc_ens)

forc_nens=len(npforc_ens)
#print(forc_nens)
#raise Exception('STOP!')

forc_ybeg,forc_yend=netCDF4.num2date(min(forc_mins),forc_times[0].units,forc_times[0].calendar).year,netCDF4.num2date(max(forc_maxs),forc_times[0].units,forc_times[0].calendar).year
forc_mbeg,forc_mend=netCDF4.num2date(min(forc_mins),forc_times[0].units,forc_times[0].calendar).month,netCDF4.num2date(max(forc_maxs),forc_times[0].units,forc_times[0].calendar).month

forc_j,forc_k,forc_date,forc_m,forc_n,forc_time=get_timestamp_number(forc_ybeg,forc_yend,forc_mbeg,forc_mend,forc_times[0].units,forc_times[0].calendar)

forc_ntime=len(forc_time)

forc_year_fraction_monthly=fractional_year_from_num2date(forc_date,forc_times[0].calendar)

#print('forc_ntime=',forc_ntime)
#print('forc_date=',forc_date)
#print('forc_time',forc_time)
#print('forc_year_fraction_monthly',forc_year_fraction_monthly)

forc_beg,forc_end=np.zeros(nforc_files,dtype=int),np.zeros(nforc_files,dtype=int) #these indices define beg/end time indices.

forc_beg_cnt,forc_end_cnt=np.zeros((forc_nens,(nforc_files/forc_nens)),dtype=int),np.zeros((forc_nens,(nforc_files/forc_nens)),dtype=int) #these indices define beg/end time indices.

#forc_nino_monthly=ma.zeros((nindices_nino,forc_nens,nforc_files,forc_ntime),dtype=float) #set them all to missing value, then assign the forecasts to the various segments in the array.\\
#forc_nino_monthly=ma.masked_equal( np.zeros((nindices_nino,forc_nens,(nforc_files/forc_nens),forc_ntime),dtype=float), 0.0) #set them all to missing value, then assign the forecasts to the various segments in the array.\\
forc_nino_monthly=ma.masked_equal( np.zeros((forc_ntime,nindices_nino,forc_nens,(nforc_files/forc_nens)),dtype=float), 0.0) #set them all to missing value, then assign the forecasts to the various segments in the array.\\

#print(forc_nino_monthly.view(ma.MaskedArray))

#raise Exception('STOP!')

forc_check=ma.zeros((nindices_nino,forc_nens,nforc_files),dtype=int) #can use this to see how the arrays are being populated or not.

#print('forc_nino_monthly.shape=',forc_nino_monthly.shape)

#forc_beg_date=date_time_stamps[0][0] #this would be for total set of times.
#forc_end_date=date_time_stamps[-1][-1] #this would be for total set of times.

forc_lat=forc_ifhs[0].variables['latitude'][:,0]
forc_lon=forc_ifhs[0].variables['longitude'][0,:]

#forc_clat=np.zeros((nforc_files),dtype=float) #if latitude range changes will need to have custom clat.
forc_clat=np.cos(forc_lat[:]*rad)

#print(indices_i[0][0],indices_i[0][1])

forc_ens_cnt=np.zeros(forc_nens,dtype=int)

for forc_i,forc_input_file in enumerate(forc_files):
  #print('forc_i,forc_input_file=',forc_i,forc_input_file)
  forc_beg_date=forc_date_time_stamps[forc_i][0]
  forc_end_date=forc_date_time_stamps[forc_i][-1]
  #print('forc_beg_date,forc_end_date=',forc_beg_date,forc_end_date)

  #print(type(forc_ens[forc_i]))
  #raise Exception('STOP!')
  
  forc_begend_time=[netCDF4.date2num(forc_beg_date,forc_times[i].units,forc_times[i].calendar), netCDF4.date2num(forc_end_date,forc_times[i].units,forc_times[i].calendar)]
  forc_loc_beg,forc_loc_end=np.where(forc_time[:]==forc_begend_time[0],1,0),np.where(forc_time[:]==forc_begend_time[1],1,0)
  
  #print('forc_loc_beg,end=',forc_loc_beg,forc_loc_end)
  #raise Exception('STOP!')
  
  forc_beg[forc_i],forc_end[forc_i]=np.argmax(forc_loc_beg),np.argmax(forc_loc_end)
  
  #print('forc_beg[forc_i],end[forc_i]=',forc_beg[forc_i],forc_end[forc_i])
  #raise Exception('STOP!')
  forc_ens_cnt[forc_ens[forc_i]-1]+=1 #this is used to put each forecast in a unique ensemble slot in the array, the array should end up with equal values if the experiment is consistent & complete.
  #print('forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1=',forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1)

  forc_beg_cnt[forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1], forc_end_cnt[forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1]=np.argmax(forc_loc_beg),np.argmax(forc_loc_end)

  for forc_k,indice in enumerate(indices_nino):

    forc_imin,forc_imax=indices_i_gn[forc_k][0],indices_i_gn[forc_k][1]
    forc_jmin,forc_jmax=indices_j_gn[forc_k][0],indices_j_gn[forc_k][1]
    #print('forc_k,forc_imin,forc_imax,forc_jmin,forc_jmax=',forc_k,forc_imin,forc_imax,forc_jmin,forc_jmax)
    
    forc_check[forc_k,forc_ens[forc_i]-1,forc_i] = forc_check[forc_k,forc_ens[forc_i]-1,forc_i] + 1 #can use this for checking. For example, onece an array is set to 1 it should not be set/reset again (no overlap).
    
    #print('forc_nino_monthly.shape=',forc_nino_monthly.shape)
    
    forc_nino_monthly[forc_beg[forc_i]:forc_end[forc_i]+1,forc_k,forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1]=np.average(np.average(forc_ifhs[forc_i].variables[dvar][:,forc_jmin:forc_jmax+1,forc_imin:forc_imax+1],axis=1,weights=forc_clat[forc_jmin:forc_jmax+1]),axis=1)
    #forc_nino_monthly[forc_k,forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1,forc_beg[forc_i]:forc_end[forc_i]+1]=np.average(np.average(forc_ifhs[forc_i].variables[dvar][:,forc_jmin:forc_jmax+1,forc_imin:forc_imax+1],axis=1,weights=forc_clat[forc_jmin:forc_jmax+1]),axis=1)
    #nb = raw_input('Press enter: ')
    #raise Exception('STOP!')
    
#print('forc_check=',forc_check)

#print('forc_ens_cnt=',forc_ens_cnt)

#print('forc_beg=',forc_beg)
#print('forc_end=',forc_end)

#print('forc_beg_cnt=',forc_beg_cnt)
#print('forc_end_cnt=',forc_end_cnt)

#raise Exception('STOP!')

#forc_nino_anomaly=forc_nino_monthly.copy() #make a copy in preparation for generating anomalies.

forc_nino_anomaly_step1=np.expand_dims(forc_nino_monthly,-1)

forc_nino_anomaly=np.tile(forc_nino_anomaly_step1,(cont_nexps))

print('forc_nino_anomaly.shape=',forc_nino_anomaly.shape)

#raise Exception('STOP!')

forc_years,forc_months,forc_days,forc_hours=[],[],[],[]
forc_yearsM1,forc_monthsM1,forc_daysM1,forc_hoursM1=[],[],[],[]
for forc_i,forc_date_now in enumerate(forc_date):
  forc_years.append(forc_date_now.year)
  forc_months.append(forc_date_now.month)
  forc_days.append(forc_date_now.day)
  forc_hours.append(forc_date_now.hour)
  forc_yearsM1.append(forc_date_now.year-1)
  forc_monthsM1.append(forc_date_now.month-1)
  forc_daysM1.append(forc_date_now.day-1)
  forc_hoursM1.append(forc_date_now.hour-1)

plot_index='nino34'
#plot_index='nino3'
#plot_index='nino4'
#plot_index='nino1+2'

forc_ens_cnt=np.zeros(forc_nens,dtype=int)

#some of these things could be put into a function.
for forc_i,forc_input_file in enumerate(forc_files):
  #print('forc_input_file=',forc_input_file)
  forc_year_beg,forc_year_end=forc_years[forc_beg[forc_i]],forc_years[forc_end[forc_i]]
  forc_month_beg,forc_month_end=forc_months[forc_beg[forc_i]],forc_months[forc_end[forc_i]]
  #print('forc_i,forc_year_beg,forc_year_end,forc_month_beg,forc_month_end=',forc_year_beg,forc_year_end,forc_month_beg,forc_month_end)
  forc_mbeg,forc_mend=1,12
  forc_ybeg,forc_yend=forc_year_beg,forc_year_end
  if(forc_month_beg!=1): forc_ybeg,forc_mbeg=forc_year_beg+1,1
  if(forc_month_end!=12): forc_yend,forc_mbeg=forc_year_end-1,12
  #print('forc_ybeg,forc_yend,forc_mbeg,forc_mend=',forc_ybeg,forc_yend,forc_mbeg,forc_mend)
  #print('forc_nino_monthly[0,forc_ens[forc_i],forc_i,forc_beg[forc_i]:forc_end[forc_i]]=',forc_nino34_monthly[0,0,forc_i,forc_beg[forc_i]:forc_end[forc_i]])
  #print('forc_years[forc_beg[forc_i]:forc_end[forc_i]+1]=',forc_years[forc_beg[forc_i]:forc_end[forc_i]+1])
  #print('forc_months[forc_beg[forc_i]:forc_end[forc_i]+1]=',forc_months[forc_beg[forc_i]:forc_end[forc_i]+1]) #just need to find first january and last december in this list to extract the indices...
  #print('cont_nino_climatology.shape=',cont_nino_climatology.shape) #cont_climatology calculated in previous cell.
  #print('cont_nino_climatology=',cont_nino_climatology)
  #print(cont_nino_climatology[0,forc_months[forc_beg[forc_i]]-1])
  #print(cont_nino_climatology[0,forc_monthsM1[forc_beg[forc_i]:forc_end[forc_i]+1]])
  forc_ens_cnt[forc_ens[forc_i]-1]+=1
  
  for forc_k,indice in enumerate(indices_nino):
    #None
    #pretty cool, each month of climatology (12) will be mapped according to values of forc_monthsM1
    #forc_nino_anomaly[forc_k,forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1,forc_beg[forc_i]:forc_end[forc_i]+1] = forc_nino_anomaly[forc_k,forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1,forc_beg[forc_i]:forc_end[forc_i]+1] - cont_nino_climatology[forc_k,forc_monthsM1[forc_beg[forc_i]:forc_end[forc_i]+1]]
    for clim_cnt in range(cont_nexps):
      forc_nino_anomaly[forc_beg[forc_i]:forc_end[forc_i]+1,forc_k,forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1,clim_cnt] = \
      forc_nino_anomaly[forc_beg[forc_i]:forc_end[forc_i]+1,forc_k,forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1,clim_cnt] - cont_nino_climatology[forc_monthsM1[forc_beg[forc_i]:forc_end[forc_i]+1],forc_k,clim_cnt]
  
  #raise Exception('STOP!')
  
  forc_year_fraction_monthly_now=forc_year_fraction_monthly[forc_beg[forc_i]:forc_end[forc_i]+1]
  #forc_zero=np.zeros(len(forc_year_fraction_monthly_now))
  #print('forc_ens[forc_i]=',forc_ens[forc_i])
  
  plt.plot(forc_year_fraction_monthly_now,forc_nino_anomaly[forc_beg[forc_i]:forc_end[forc_i]+1,indices_nino.index(plot_index),forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1,0],color=colors[forc_ens[forc_i]-1])
  #plt.plot(forc_year_fraction_monthly_now,forc_nino_monthly[indices_nino.index(plot_index),forc_ens[forc_i]-1,forc_ens_cnt[forc_ens[forc_i]-1]-1,forc_beg[forc_i]:forc_end[forc_i]+1],color=colors[forc_ens[forc_i]-1])

#  print('Times go from ',min(forc_mins),'to',max(forc_maxs),'or',
#    netCDF4.num2date(min(forc_mins),forc_times[0].units,forc_times[0].calendar),'to',netCDF4.num2date(max(forc_maxs),forc_times[0].units,forc_times[0].calendar))

#year_min=netCDF4.num2date(min(forc_mins),forc_times[0].units,forc_times[0].calendar)
#year_max=netCDF4.num2date(max(forc_maxs),forc_times[0].units,forc_times[0].calendar)
all_years=np.linspace(year_min.year,year_max.year,20)
forc_zero=np.zeros(len(all_years))
plt.plot(all_years,forc_zero,color='black',linestyle=':')

plt.title('Monthly anomalies')
plt.xlabel('Year')
plt.ylabel(indices_label[indices_nino.index(plot_index)]+' ($^o$C)')
plt.show()

print('END')

# Process observational data

In [ ]:
# print('BEGIN')

import matplotlib.ticker as ticker

nmy=12

#del(generate_daily_month_indices)
#from decadal_diag import generate_daily_month_indices

ivar_str_ncep2='skt'
#ybeg_data_ncep2=2000 #2000;assume 12 months in every year.
#yend_data_ncep2=2016 #2016;assume 12 months in every year.

grid_label='ncep2'
ncep2_files_string='/short/v14/mac599/ncep2/skt.sfc.gauss.????.nc'
ncep2_files=sorted(glob.glob(ncep2_files_string))
nncep2_files=len(ncep2_files)
print('ncep2 input_files ('+str(nncep2_files)+') =',ncep2_files)

ifhN_ncep2=netCDF4.MFDataset(ncep2_files)
ifh0_ncep2=netCDF4.MFDataset(ncep2_files[0])

time_daily_ncep2=ifhN_ncep2.variables['time']
#lat=ifh0.variables['lat']
#lon=ifh0.variables['lon']

lat_ncep2=ifh0_ncep2.variables['lat'][:]
lon_ncep2=ifh0_ncep2.variables['lon'][:]

ntime_daily_ncep2=len(time_daily_ncep2)
#print('lat=',lat)

ifh0_ncep2.close()

#print('rad=',rad)
clat_ncep2=np.cos(lat_ncep2[:]*rad)

#time_daily_ncep2_calendar='julian'
time_daily_ncep2_calendar='proleptic_gregorian'

date_time_stamp_daily_ncep2=netCDF4.num2date(time_daily_ncep2[:],time_daily_ncep2.units,time_daily_ncep2_calendar)

year_fraction_daily_ncep2=fractional_year_from_num2date(date_time_stamp_daily_ncep2,time_daily_ncep2_calendar)

nino_daily_ncep2=ma.zeros((ntime_daily_ncep2,nindices_nino),dtype=float)
#raise Exception('STOP!')
for ncep2_k,indice in enumerate(indices_nino):
  #ncep2_k=indices_nino.index(indice)
  ncep2_imin,ncep2_imax=indices_i_ncep2[indices_nino.index(indice)][0],indices_i_ncep2[indices_nino.index(indice)][1]
  ncep2_jmin,ncep2_jmax=indices_j_ncep2[indices_nino.index(indice)][0],indices_j_ncep2[indices_nino.index(indice)][1]
  #print('indices.index(indice),imin_ncep2,imax_ncep2,jmin_ncep2,jmax_ncep2=',indices.index(indice),imin_ncep2,imax_ncep2,jmin_ncep2,jmax_ncep2)
  #print('lat_ncep2,lon_ncep2=',lat_ncep2[jmin_ncep2:jmax_ncep2],lon_ncep2[imin_ncep2:imax_ncep2])

  nino_daily_ncep2[:,ncep2_k]=np.average(np.average(ifhN_ncep2.variables['skt'][:,ncep2_jmin:ncep2_jmax,ncep2_imin:ncep2_imax],axis=1,weights=clat_ncep2[ncep2_jmin:ncep2_jmax]),axis=1)

#indices_i_gn,indices_j_gn=[[122,152],[122,152],[122,152]],[[110,160],[110,160],[110,160]] #check this, whether I need +1 also...
#raise Exception('STOP!')
#print('year_fraction_ncep2=',year_fraction_monthly_ncep2)

%matplotlib inline

fix,ax=plt.subplots()
for ncep2_k,indice in enumerate(indices_nino):
  ax.plot(year_fraction_daily_ncep2,nino_daily_ncep2[:,indices_nino.index(indice)]-273.15,label=indices_label[indices_nino.index(indices_nino[ncep2_k])])
  ax.xaxis.set_major_locator(ticker.MultipleLocator(2.0))
legend=ax.legend(loc='lower right',shadow=False,fontsize='x-large')
plt.title('Daily values (NCEP2)')
plt.xlabel('Year')
plt.ylabel('$^o$C')
plt.grid(True,linestyle='-')
plt.show()

what_to_keep='all_times'
#what_to_keep='all_but_first'
#what_to_keep='all_but_last'
#what_to_keep='all_but_first_and_last'

if(what_to_keep=='all_times'):
  #all times kept:
  date_time_stamp_daily_new_ncep2=date_time_stamp_daily_ncep2
  nino_daily_new_ncep2=nino_daily_ncep2
  year_fraction_daily_new_ncep2=year_fraction_daily_ncep2
  
elif(what_to_keep=='all_but_first'):
  #remove first day:
  date_time_stamp_daily_new_ncep2=date_time_stamp_daily_ncep2[1::]
  nino_daily_new_ncep2=nino_daily_ncep2[1::,:]
  year_fraction_daily_new_ncep2=year_fraction_daily_ncep2[1::]
  
elif(what_to_keep=='all_but_last'):
  #remove last day:
  date_time_stamp_daily_new_ncep2=date_time_stamp_daily_ncep2[0:-1]
  nino_daily_new_ncep2=nino_daily_ncep2[0:-1,:]
  year_fraction_daily_new_ncep2=year_fraction_daily_ncep2[0:-1]

elif(what_to_keep=='all_but_first_and_last'):
  #remove first and last day:
  date_time_stamp_daily_new_ncep2=date_time_stamp_daily_ncep2[1:-1]
  nino_daily_new_ncep2=nino_daily_ncep2[1:-1,:]
  year_fraction_daily_new_ncep2=year_fraction_daily_ncep2[1:-1]
  
else:
  raise Exception('That what_to_keep option doesnt exist.')

#print('len(date_time_stamp_daily_new_ncep2)=',len(date_time_stamp_daily_new_ncep2))

daily_month_indice_beg_ncep2,daily_month_indice_end_ncep2,daily_year_beg_ncep2,daily_year_end_ncep2,daily_month_beg_ncep2,daily_month_end_ncep2,daily_day_beg_ncep2,daily_day_end_ncep2,beg_month_partial_ncep2,end_month_partial_ncep2 = \
  generate_daily_month_indices(date_time_stamp_daily_new_ncep2,time_daily_ncep2.units,time_daily_ncep2_calendar,24)

#raise Exception('STOP!')

nino_monthly_ncep2=ma.zeros((len(daily_month_indice_beg_ncep2),nindices_nino),dtype=float)

#raise Exception('STOP!')

for k,indice in enumerate(indices_nino):
  for month in range(0,len(daily_month_indice_beg_ncep2)):
    nino_monthly_ncep2[month,k]=np.average(nino_daily_new_ncep2[daily_month_indice_beg_ncep2[month]:daily_month_indice_end_ncep2[month]+1,k])

#raise Exception('STOP!')

year_fraction_monthly_ncep2=ma.zeros(len(daily_month_indice_beg_ncep2))
for month in range(0,len(daily_month_indice_beg_ncep2)):
  year_fraction_monthly_ncep2[month]=np.average(year_fraction_daily_new_ncep2[daily_month_indice_beg_ncep2[month]:daily_month_indice_end_ncep2[month]+1])

num_stamp_new_ncep2=netCDF4.date2num(date_time_stamp_daily_new_ncep2,time_daily_ncep2.units,time_daily_ncep2_calendar)

num_stamp_monthly_ncep2=np.zeros(len(daily_month_indice_beg_ncep2))
#print('num_stamp_monthly_ncep2.shape=',num_stamp_monthly_ncep2.shape)

for month in range(0,len(daily_month_indice_beg_ncep2)):
  num_stamp_monthly_ncep2[month]=np.average(num_stamp_new_ncep2[daily_month_indice_beg_ncep2[month]:daily_month_indice_end_ncep2[month]+1])

date_time_stamp_monthly_ncep2=netCDF4.num2date(num_stamp_monthly_ncep2,time_daily_ncep2.units,time_daily_ncep2_calendar)

#raise Exception('STOP!')
  
EndOption=1 #use months with no days missing at begin and end months (i.e. discard partial months if they exist).
#EndOption=2 #use months with days missing at both begin and end months (i.e. don't discard partial months if they exist @ beg/end).

print('beg,end_month_partial_ncep2=',beg_month_partial_ncep2,end_month_partial_ncep2)

if(EndOption==1):
  print('Discarding beg&/end month if they exist.')
  if(beg_month_partial_ncep2 or end_month_partial_ncep2):
    if(beg_month_partial_ncep2 and end_month_partial_ncep2):
      print('type#1')
      nino_monthly_new_ncep2=nino_monthly_ncep2[1:-1,:]
      year_fraction_monthly_new_ncep2=year_fraction_monthly_ncep2[1:-1]
      date_time_stamp_monthly_new_ncep2=date_time_stamp_monthly_ncep2[1:-1]
      
    elif(not beg_month_partial_ncep2 and end_month_partial_ncep2):
      print('type#2')
      nino_monthly_new_ncep2=nino_monthly_ncep2[0:-1,:]
      year_fraction_monthly_new_ncep2=year_fraction_monthly_ncep2[0:-1]
      date_time_stamp_monthly_new_ncep2=date_time_stamp_monthly_ncep2[0:-1]

    elif(beg_month_partial_ncep2 and not end_month_partial_ncep2):
      print('type#3')
      nino_monthly_newnino_ncep2=nino_monthly_ncep2[1::,:]
      year_fraction_monthly_new_ncep2=year_fraction_monthly_ncep2[1::]
      date_time_stamp_monthly_new_ncep2=date_time_stamp_monthly_ncep2[1::]

    else:
      raise SystemExit('Shouldnt get here:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  else:
    print('type#4')
    nino_monthly_new_ncep2=nino_monthly_ncep2
    year_fraction_monthly_new_ncep2=year_fraction_monthly_ncep2
    date_time_stamp_monthly_new_ncep2=date_time_stamp_monthly_ncep2
    
elif(EndOption==2):
  print('Keeping beg/end month or both.')
  nino_monthly_new_ncep2=nino_monthly_ncep2
  year_fraction_monthly_new_ncep2=year_fraction_monthly_ncep2
  date_time_stamp_monthly_new_ncep2=date_time_stamp_monthly_ncep2
  
else:
  raise SystemExit('EndOption can be only 1 or 2:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
#raise Exception('STOP!')

%matplotlib inline

#print(nino_monthly_new)

#print('year_fraction_monthly_new=',year_fraction_monthly_new)

plot_index='nino34'
#plot_index='nino1+2'
#plot_index='nino4'

fix,ax=plt.subplots()
ax.plot(year_fraction_monthly_new_ncep2,nino_monthly_new_ncep2[:,indices_nino.index(plot_index)]-273.15,color='blue',label='mon_from_day')
legend=ax.legend(loc='lower left',shadow=False,fontsize='large')
ax.xaxis.set_major_locator(ticker.MultipleLocator(2.0))
plt.title('Monthly values (NCEP2)')
plt.xlabel('Year')
plt.ylabel(indices_label[indices_nino.index(plot_index)] +'($^o$C)')
plt.grid(True,linestyle='-')
plt.show()

#raise Exception('STOP!')

#print('date_time_stamp_monthly_new_ncep2=',date_time_stamp_monthly_new_ncep2)
ybeg_ncep2=date_time_stamp_monthly_new_ncep2[0].year
yend_ncep2=date_time_stamp_monthly_new_ncep2[-1].year

ydiff_monthly_ncep2=yend_ncep2-ybeg_ncep2+1

MissingMonths_ncep2=False

first_month_ncep2=date_time_stamp_monthly_new_ncep2[0].month
last_month_ncep2=date_time_stamp_monthly_new_ncep2[-1].month

#print('ybeg,yend_ncep2=',ybeg_ncep2,yend_ncep2)

#print('first,last_month_ncep2=',first_month_ncep2,last_month_ncep2)

missing_months_beg_ncep2,missing_months_end_ncep2=0,0

cyear_beg_skip_ncep2,cyear_end_skip_ncep2=0,1

if(first_month_ncep2!=1):
  missing_months_beg_ncep2=12-first_month_ncep2
  cyear_beg_skip_ncep2=1
  MissingMonths_ncep2=True

if(last_month_ncep2!=12):
  missing_months_end_ncep2=12-last_month_ncep2
  cyear_end_skip_ncep2=2
  MissingMonths_ncep2=True
  
if(MissingMonths_ncep2):
  print('There are missing months in the set. '+str(missing_months_beg_ncep2)+' at beginning and '+str(missing_months_end_ncep2)+' at end.')
  print('Currently years with missing months are not used in generating long term monthly climatology.')
  print('And missing months will be set to missing in the final time-series.')
  ts_beg_ncep2,ts_end_ncep2,ts_avg_ncep2,dt_beg_ncep2,dt_end_ncep2,dt_avg_ncep2=get_timestamp_number(ybeg_ncep2,yend_ncep2,1,12,time_ncep2.units,time_ncep2_calendar)
  year_fraction_monthly_full_ncep2=fractional_year_from_num2date(ts_avg_ncep2,time_ncep2_calendar)
  
  nino_monthly_full_ncep2=ma.masked_all((ydiff_monthly_ncep2*nmy,nindices_nino),dtype=float) #ensure missing months are masked out.
  last_month_index_ncep2=ydiff_monthly_ncep2*nmy-last_month_ncep2
  nino_monthly_full_ncep2[first_month_ncep2-1:last_month_index_ncep2,:]=nino_monthly_ncep2
else:
  print('All years have 12 months.')
  
  nino_monthly_full_ncep2=nino_monthly_ncep2.copy()
  year_fraction_monthly_full_ncep2=year_fraction_monthly_ncep2.copy()

  print('nino_monthly_full_ncep2.shape=',nino_monthly_full_ncep2.shape)

nino_climatology_ncep2,nino_monthly_anomaly_ncep2 = calculate_monthly_climatology_anomaly_from_monthly(nino_monthly_full_ncep2)

#raise Exception('STOP!')

%matplotlib inline

zero_ncep2=np.zeros(len(year_fraction_monthly_full_ncep2))
fix,ax=plt.subplots()
for ncep2_k,indice in enumerate(indices_nino):
  ax.plot(year_fraction_monthly_full_ncep2,nino_monthly_anomaly_ncep2[:,ncep2_k],label=indices_label[indices_nino.index(indices_nino[ncep2_k])])
plt.plot(year_fraction_monthly_full_ncep2,zero_ncep2,color='black')

legend=ax.legend(loc='upper left',shadow=False,fontsize='x-large')
ax.xaxis.set_major_locator(ticker.MultipleLocator(2.0))
plt.title('Monthly anomalies (NCEP2)')
plt.xlabel('Year')
plt.ylabel('$^o$C')
plt.grid(True,linestyle='-')
plt.show()

print('END')

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import os
from mpl_toolkits.basemap import Basemap

%matplotlib inline

#https://www.youtube.com/watch?v=6SHnmho7zCs

def plot_map_box(ind,indices_label,indices_nino,lats_nino,lons_nino):
  
  colors=['black','red','green','blue','orange','brown','purple','pink']

  ind_list=[]
  for f,ff in enumerate(ind):
    ind_list.append(indices_nino.index(ind[f])) 
  
  map = Basemap(projection='cyl', 
  llcrnrlat=-30, llcrnrlon=100, 
  urcrnrlat=30, urcrnrlon=320,
  lat_0=0, lon_0=180)

  map.drawmapboundary() #fill_color='aqua'
  map.fillcontinents() #color='coral',lake_color='aqua'
  map.drawcoastlines()

  #print('len(lons_nino)=',len(lons_nino))

  for i,ii in enumerate(ind_list):
    x,y = map(lons_nino[ii], lats_nino[ii])
    map.plot(x, y, marker=None,color=colors[i],linewidth=1)
    xmid,ymid = map((lons_nino[ii][0]+lons_nino[ii][1])/2,(lats_nino[ii][0]+lats_nino[ii][2])/2)
    plt.text(xmid,ymid,indices_nino[ii],fontsize=6,horizontalalignment='left',verticalalignment='center')  

  plt.show()

def plot_xy_climatology(ind,obs,mod,clim,indices_label,obs_toggle):

  colors=['black','red','green','blue','orange','brown','purple','pink']

  ind_list=[]
  for f,ff in enumerate(ind):
    ind_list.append(indices_nino.index(ind[f])) 

  #print('indices_label=',indices_label)
  #print('ind=',ind)
  #print('ind_list=',ind_list)
  
  fix,ax=plt.subplots()
  if(obs_toggle):
    plt.title('Monthly climatology (obs:dashed)')
  else:
    plt.title('Monthly climatology')

  plt.xlabel('Month')
  plt.ylabel('$^o$C')
  plt.xticks(range(12), ['J','F','M','A','M','J','J','A','S','O','N','D'], rotation='horizontal')

  for j in ind_list:
    ax.plot(range(12),mod[:,j,clim],color=colors[j],label=indices_label[j])
    if(obs_toggle):
      ax.plot(range(12),obs[:,j]-273.15,color=colors[j],linestyle=':')
  plt.grid(True,linestyle='-')
  legend=ax.legend(loc='lower right',shadow=False,fontsize='small',title='model indice')
  plt.show()
  
def plot_xy_ensemble(mod_month,mod_anom,mod_time,obs_month,obs_anom,obs_time,indices_nino,ensembles, \
  ind,ens,forc,clim,indices_label,forc_beg,forc_end,forc_beg_cnt,forc_end_cnt,files,anom_toggle, \
  ensemble_average_toggle,obs_toggle,print_file):
  #print('data.shape=',data.shape)
  #print('ind,indices_nino=',ind,indices_nino)
  #print('ens,ensembles=',ens,ensembles)
  #x=range(len(data[0,0,0,:]))
  #print('data.shape=',data.shape)
  #print('forc_beg=',forc_beg)
  #print('x=',x)
  #plt.plot(time,data[ind,ens,0,0:24])
  
  colors=['black','red','green','blue','orange','brown','purple','pink']

  #print('Current Working Directory=',os.getcwd())
  
  #print('ensemble_average_toggle=',ensemble_average_toggle)

  #if(print_file==None):
  #  print('No print file.')
  #else:
  #  print('print_file=',print_file)
  #print('jjj=',jjj)
  if(anom_toggle):
    mod_data=mod_anom[:,:,:,:,clim]
    obs_data=obs_anom
  else:
    mod_data=mod_month
    obs_data=obs_month-273.15
  #print('obs_data=',obs_data)
  
  ens_list=[]
  for e,ee in enumerate(ens):
    ens_list.append(ensembles.index(ens[e]))
    
  ind_list=[]
  for f,ff in enumerate(ind):
    ind_list.append(indices_nino.index(ind[f])) 
    
  fix,ax=plt.subplots()
  #for i in range(len(files)/len(ensembles)):
  for i in forc:
    #print('i=',i)
    ival=int(i.split(':')[0])
    #print('ival=',ival)
    #time_now=time[forc_beg[i]:forc_end[i]+1]
    for j in ind_list:
      #print('i,j,k=',i,j,k)
      for k in ens_list:
          mod_time_now_test=mod_time[forc_beg_cnt[k,ival]:forc_end_cnt[k,ival]+1]
      #print('time_now=',time_now)
          if(i==forc[0] and j==ind_list[0]):
            legend_label=str(k+1)
          else:
            legend_label=None
          ax.plot(mod_time_now_test,mod_data[forc_beg_cnt[k,ival]:forc_end_cnt[k,ival]+1,j,k,ival],color=colors[k],label=legend_label) #indice, ensemble, forecast, date/time
          if(k==ens_list[-1] and ensemble_average_toggle):
            #xxx=np.average(data[j,ens_list,ival,forc_beg_cnt[k,ival]:forc_end_cnt[k,ival]+1],axis=0)
            #print(xxx.shape)
            ax.plot(mod_time_now_test,np.average(mod_data[forc_beg_cnt[k,ival]:forc_end_cnt[k,ival]+1,j,ens_list,ival],axis=0),color='pink',linewidth=5,label='EA')
          if(k==ens_list[-1] and i==forc[-1] and obs_toggle):
            #print('hello')
            ax.plot(obs_time,obs_data[:,j],color='lightblue',label='ncep2')
  legend=ax.legend(loc='lower right',shadow=False,fontsize='small',title='ensemble')
  if(anom_toggle):
    plt.title('Monthly anomalies')
    plt.ylim([-6,6])
  else:
    plt.title('Monthly values')
  plt.xlabel('Year')
  #plt.ylabel(indices_label[ind]+' ($^o$C)')
  plt.ylabel('$^o$C')
  #plt.ylim([-6,6])
  #plt.xlim([x[0],x[-1]])
  plt.xlim([mod_time[0],mod_time[-1]])
  plt.grid(True,linestyle='-')
  if(anom_toggle):
   cont_zero=np.zeros(len(mod_time))
   plt.plot(mod_time,cont_zero,color='black',linestyle=':')
  
  if(print_file==None or print_file==''):
    None
  else:
    print('Print to file ',os.getcwd()+'/'+print_file+'.png')
    dummy=plt.savefig(print_file)
    print_file=None
  #return()
  plt.show()

#print('datetime_uniq=',datetime_uniq+range(24))
datetime_uniq_xtra=[str(i)+': '+datetime_uniq[i] for i,x in enumerate(datetime_uniq)]
#print('datetime_uniq_xtra=',datetime_uniq_xtra)
#print('len(datetime_uniq_xtra)=',len(datetime_uniq_xtra))

clim_list=[int(i) for i in range(cont_nexps)]

ensembles=[str(x) for x in npforc_ens]
indice_nino_multi = widgets.SelectMultiple(description='nino indice',options=indices_nino,value=indices_nino)
ensemble_multi = widgets.SelectMultiple(description='ensemble',options=ensembles,value=ensembles)
forecast_multi = widgets.SelectMultiple(description='forecast',options=datetime_uniq_xtra,value=datetime_uniq_xtra,rows=10)
climatology_select = widgets.Select(description='climatology',options=clim_list,value=clim_list[0])
anom_toggle=widgets.ToggleButton(description='Show Anoms.',value=False,button_style='danger')

obs_toggle=widgets.ToggleButton(description='Show Obs.',value=False,button_style='danger')

ensemble_average_toggle=widgets.ToggleButton(description='Show E.A.',value=False,button_style='danger')
style = {'description_width': 'initial'}
print_file=widgets.Text(value=None,placeholder='Print file name (delete to stop printing it):',continuous_update=False,style=style)

#container=widgets.HBox([indice_nino_multi,ensemble_multi,forecast_multi,anom_toggle])
#display(container)
  
#raise Exception('STOP!')
#print(print_file.keys)
#print(anom_toggle.keys)

mod_month_to_plot=forc_nino_monthly.copy()
mod_anom_to_plot=forc_nino_anomaly.copy()

obs_month_to_plot=nino_monthly_full_ncep2.copy()
obs_anom_to_plot=nino_monthly_anomaly_ncep2.copy()

dummy1=widgets.interact(plot_xy_ensemble, mod_time=widgets.fixed(forc_year_fraction_monthly), mod_month=widgets.fixed(mod_month_to_plot), mod_anom=widgets.fixed(mod_anom_to_plot), \
  obs_time=widgets.fixed(year_fraction_monthly_full_ncep2), obs_month=widgets.fixed(obs_month_to_plot), obs_anom=widgets.fixed(obs_anom_to_plot), \
  indices_label=widgets.fixed(indices_label), forc_beg=widgets.fixed(forc_beg), forc_end=widgets.fixed(forc_end), \
  forc_beg_cnt=widgets.fixed(forc_beg_cnt), forc_end_cnt=widgets.fixed(forc_end_cnt), files=widgets.fixed(forc_files), \
  ensembles=widgets.fixed(ensembles), indices_nino=widgets.fixed(indices_nino), \
  ind=indice_nino_multi, ens=ensemble_multi, forc=forecast_multi, clim=climatology_select, anom_toggle=anom_toggle, ensemble_average_toggle=ensemble_average_toggle, obs_toggle=obs_toggle, print_file=print_file)

dummy2=widgets.interact(plot_xy_climatology, \
  obs=widgets.fixed(nino_climatology_ncep2), mod=widgets.fixed(cont_nino_climatology), \
  indices_label=widgets.fixed(indices_label), \
  ind=indice_nino_multi, clim=climatology_select, obs_toggle=obs_toggle)

dummy3=widgets.interact(plot_map_box, indices_label=widgets.fixed(indices_label), indices_nino=widgets.fixed(indices_nino), lats_nino=widgets.fixed(lats_nino), lons_nino=widgets.fixed(lons_nino), \
  ind=indice_nino_multi, obs_toggle=obs_toggle)

In [ ]:
for i in [0,3,5]:
  print(i)

In [ ]:
from __future__ import print_ function
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

#%matplotlib nbagg
#%matplotlib notebook
%matplotlib inline

print('forc_nino_anomaly.shape=',forc_nino_anomaly.shape) #indice, ensemble, forecast, date/time

#indices_nino=['nino34','nino3','nino4']

print('indices_nino=',indices_nino)

indices_nino_dict={}
for i,d in enumerate(indices_nino):
  indices_nino_dict[d]=i

print('indices_nino_dict=',indices_nino_dict)

ensembles=[str(x) for x in npforc_ens]

print('ensembles=',ensembles)

print('forc_nens=',forc_nens)

#fig, ax = plt.subplots(1,figsize=(10,4))
#plt.suptitle('Monthly anomalies')

def test_plot(ensembles, indices_nino):
  #from IPython.display import display
  print('ensembles,indces_nino=',ensembles,indices_nino)
  #display(ensemble_range, indice_nino_range)
  #ax.clear()
  #all_years=np.linspace(2001,2020,20)
  #line=all_years-2001+.1
  #t = np.arange(0., 5., 0.2)
  #ax.plot( t, t**2, 'bs')
  #ax.plot(all_years,line,color='black',linestyle=':')
  #ax.legend(loc=1)
  #plt.show()
  plt.figure(2)
  x=np.linspace(-10,10,num=100)
  plt.plot(x,x**ensembles+ensembles*10)
  plt.show()


#ensemble_range = widgets.SelectMultiple(description='ensemble',options=ensembles,value=ensembles,disabled=False)

#indice_nino_range = widgets.SelectMultiple(description='nino indice',options=indices_nino,value=indices_nino,disabled=False)

#display(ensemble_range, indice_nino_range)

#interactive_plot=widgets.interactive(interactive_plot, ensembles=(0,forc_nens-1,1), indices_nino={'nino34':0, 'nino3':1, 'nino4':2})
testit=widgets.interactive(test_plot, ensembles=(0,forc_nens-1,1), indices_nino=indices_nino_dict)

#print(testit.value)

#del(interactive_plot)
#ensemble_range.observe(update_plot, names='value')

#indice_nino_range.observe(update_plot, names='value')

testit
#interactive_plot

#forc_nino_anomaly

In [ ]:
from __future__ import print_function

%matplotlib inline

print('npforc_ens=',npforc_ens)

ensembles=[str(x) for x in npforc_ens]
print('ensembles=',ensembles)

import ipywidgets as widgets

#print(widgets.Button.on_click.__doc__)

ensemble_range = widgets.SelectMultiple(description='ensemble',options=ensembles,value=ensembles)

#print(ensemble_range.keys)

#indice_nino_range = widgets.SelectMultiple(description='nino indice',options=['nino34','nino3'],value=['nino34'])
indice_nino_range = widgets.SelectMultiple(description='nino indice',options=indices_nino,value=indices_nino)

container=widgets.HBox([ensemble_range,indice_nino_range])

display(container)
#display(ensemble_range, indice_nino_range)
#display(indice_nino_range)

from IPython.display import display

#def plotit():
fig, ax = plt.subplots(1,figsize=(10,4))

def on_value_change(change):
  #ax.clear()
  plt.figure(2)
  x=np.linspace(-10,10,num=100)
  plt.plot(x,x**3+1*10)
  plt.show()
  print('hello')
  print('old',change['old'])
  print('new',change['new'])
  print('name',change['name'])

ensemble_range.observe(on_value_change, names='value')

indice_nino_range.observe(on_value_change, names='value')

print('ensemble_range=',ensemble_range.value)

print('indice_nino_range=',indice_nino_range.value)

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

In [ ]:













a=[0,1,2,3,4,0,1,2,3]
b=np.array([10,20,30,40,50])
print(b[a])

# Process/Plot assimilation run

In [ ]:
print('BEGIN.')

import math

rad = 4.0*math.atan(1.0)/180.0
#print('rad=',rad)

os.chdir(rundir)

grid_label='gr2'
grid_label='gn'
assim_files_string='CMIP6/CMIP/CSIRO/CAFE-1-0/historical/r1i1p2f1/Omon/'+dvar+'/'+grid_label+'/v20171025/'+dvar+'_Omon_historical_CAFE-1-0_r1i1p2f1_'+grid_label+'_??????-??????.nc'
assim_files=glob.glob(assim_files_string)

print('assim input_files ('+str(len(assim_files))+') =',assim_files)

assim_input_file=assim_files[5]

#print(datetime.datetime.fromtimestamp(os.stat(input_file).st_mtime))
print('assim input file=',assim_input_file,'time=',datetime.datetime.fromtimestamp(os.stat(assim_input_file).st_mtime))

assim_ifh0=netCDF4.Dataset(assim_input_file) #choose if more than 1.

assim_time=assim_ifh0.variables['time']
assim_lat=assim_ifh0.variables['latitude'][:,0]
assim_lon=assim_ifh0.variables['longitude'][0,:]
assim_clat=np.cos(assim_lat[:]*rad)

#print('lat=',lat[122:152])
#print('lon=',lon[110:160])
#print('clat=',clat)
#break

assim_nino34_monthly=np.average(np.average(assim_ifh0.variables[dvar][:,122:152,110:160],axis=1,weights=assim_clat[122:152]),axis=1) #need to add in area weighting strictly

'''
print('time.units=',time.units)
print('time.calendar=',time.calendar)
print('time=',time)
print('time[:]=',time[:])
'''

assim_date_time_stamp=netCDF4.num2date(assim_time[:],assim_time.units,assim_time.calendar)

'''
print('date_time_stamp=',date_time_stamp)
num_stamp=netCDF4.date2num(date_time_stamp,time.units,time.calendar)
print('num_stamp=',num_stamp)
print('year_fraction=',year_fraction)
'''
assim_year_fraction_monthly=fractional_year_from_num2date(assim_date_time_stamp,assim_time.calendar)

%matplotlib inline

plt.title('Monthly values')
plt.plot(assim_year_fraction_monthly,assim_nino34_monthly[:])
plt.xlabel('Year')
plt.ylabel('Ni$\~{n}$o34 ($^o$C)')
plt.show()
print('END')

In the following monthly values will be calculated and then plotted.

From the monthly values now we can calculate long term monthly climatologies and the month-to-month anomalies, and then plot the time-series. These are generally of more interest than the full monthly values.

Want to generalise climatology, anomaly generation so that any data product can be processed (e.g. (time), (time, lat, lon), (time, depth, lat, lon), (time, lev, lat, lon).

Also want to pad out years with missing months.

In [ ]:
print('BEGIN')

ybeg=date_time_stamp[0].year
yend=date_time_stamp[-1].year

ydiff=yend-ybeg+1
nmy=12

MissingMonths=False

first_month=date_time_stamp[0].month
last_month=date_time_stamp[-1].month

missing_months_beg,missing_months_end=0,0

cyear_beg_skip,cyear_end_skip=0,1

if(first_month!=1):
  missing_months_beg=12-first_month
  cyear_beg_skip=1
  MissingMonths=True

if(last_month!=12):
  missing_months_end=12-last_month
  cyear_end_skip=2
  MissingMonths=True
  
if(MissingMonths):
  print('There are missing months in the set. '+str(missing_months_beg)+' at beginning and '+str(missing_months_end)+' at end.')
  print('Currently years with missing months are not used in generating long term monthly climatology.')
  print('And missing months will be set to missing in the final time-series.')
  ts_beg,ts_end,ts_avg,dt_beg,dt_end,dt_avg=get_timestamp_number(ybeg,yend,1,12,time.units,time.calendar)
  year_fraction_monthly_full=fractional_year_from_num2date(ts_avg,time.calendar)
  
  nino34_monthly_full=ma.masked_all(ydiff*nmy,dtype=float) #ensure missing months are masked out.
  last_month_index=ydiff*nmy-last_month
  nino34_monthly_full[first_month-1:last_month_index]=nino34_monthly
else:
  print('All years have 12 months.')
  
  nino34_monthly_full=nino34_monthly
  year_fraction_monthly_full=year_fraction_monthly
  
nino34_monthly_reshaped=np.reshape(nino34_monthly_full,new_monthly_array_shape(nino34_monthly.shape,ydiff,nmy)) #check this works on large multi-dimensional arrays.

'''
#print('nino34_monthly_reshaped.shape=',nino34_monthly_reshaped.shape)
#print('cyear_beg,end_skip=',cyear_beg_skip,cyear_end_skip)
#print('ydiff=',ydiff)
#j=nino34_monthly_reshaped[cyear_beg_skip:-cyear_end_skip]
#print('j=',j)
'''

climatology=np.average(nino34_monthly_reshaped[cyear_beg_skip:-cyear_end_skip],axis=0) #average over full years only, this could be an option as could use for all years present.
nino34_monthly_climatology=np.expand_dims(climatology,0)
nino34_monthly_climatology=np.tile(nino34_monthly_climatology,(ydiff,1))
nino34_monthly_climatology_flat=nino34_monthly_climatology.flatten()
nino34_monthly_anomaly=nino34_monthly_full-nino34_monthly_climatology_flat

%matplotlib inline

zero=np.zeros(len(year_fraction_monthly_full))
plt.plot(year_fraction_monthly_full,nino34_monthly_anomaly)
plt.plot(year_fraction_monthly_full,zero)

plt.title('Monthly anomalies')
plt.xlabel('Year')
plt.ylabel('Ni$\~{n}$o34 ($^o$C)')
plt.show()

#raise Exception('STOP!')
print('END')

In [ ]:
from datetime import date

# random Person
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    @staticmethod
    def fromFathersAge(name, fatherAge, fatherPersonAgeDiff):
        return Person(name, date.today().year - fatherAge + fatherPersonAgeDiff)

    @classmethod
    def fromBirthYear(cls, name, birthYear):
        return cls(name, date.today().year - birthYear)

    def display(self):
        print(self.name + "'s age is: " + str(self.age))

class Man(Person):
    sex = 'Male'

man = Man.fromBirthYear('John', 1985)
print(isinstance(man, Man))

man1 = Man.fromFathersAge('John', 1965, 20)
print(isinstance(man1, Man))

In [ ]:
man



In [ ]:
man1

In [ ]:
man.display()

In [ ]:
man1.display()

In [ ]:
class Person:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self): # instance method
        # instance object accessible through self
        return "%s %s" % (self.name, self.surname)

    @classmethod
    def allowed_titles_starting_with(cls, startswith): # class method
        # class or instance object accessible through cls
        return [t for t in cls.TITLES if t.startswith(startswith)]

    @staticmethod
    def allowed_titles_ending_with(endswith): # static method
        # no parameter for class or instance object
        # we have to use Person directly
        return [t for t in Person.TITLES if t.endswith(endswith)]


jane = Person("Jane", "Smith")

print(jane.fullname())

print(jane.allowed_titles_starting_with("M"))
print(Person.allowed_titles_starting_with("M"))

print(jane.allowed_titles_ending_with("s"))
print(Person.allowed_titles_ending_with("s"))

In [ ]:
#http://python-textbok.readthedocs.io/en/1.0/Object_Oriented_Programming.html
#following fails, possibly need to run python3.

class Person:
    def __init__(self, name, surname, number):
        self.name = name
        self.surname = surname
        self.number = number


class Student(Person):
    UNDERGRADUATE, POSTGRADUATE = range(2)

    def __init__(self, student_type, *args, **kwargs):
        self.student_type = student_type
        self.classes = []
        super(Student, self).__init__(*args, **kwargs)

    def enrol(self, course):
        self.classes.append(course)


class StaffMember(Person):
    PERMANENT, TEMPORARY = range(2)

    def __init__(self, employment_type, *args, **kwargs):
        self.employment_type = employment_type
        super(StaffMember, self).__init__(*args, **kwargs)


class Lecturer(StaffMember):
    def __init__(self, *args, **kwargs):
        self.courses_taught = []
        super(Lecturer, self).__init__(*args, **kwargs)

    def assign_teaching(self, course):
        self.courses_taught.append(course)


jane = Student(Student.POSTGRADUATE, "Jane", "Smith", "SMTJNX045")
jane.enrol(a_postgrad_course)

bob = Lecturer(StaffMember.PERMANENT, "Bob", "Jones", "123456789")
bob.assign_teaching(an_undergrad_course)


In [ ]:
class Song:

    def __init__(self, title, artist, album, track_number):
        self.title = title
        self.artist = artist
        self.album = album
        self.track_number = track_number

        artist.add_song(self)


class Album:

    def __init__(self, title, artist, year):
        self.title = title
        self.artist = artist
        self.year = year

        self.tracks = []

        artist.add_album(self)

    def add_track(self, title, artist=None):
        if artist is None:
            artist = self.artist

        track_number = len(self.tracks)

        song = Song(title, artist, self, track_number)

        self.tracks.append(song)


class Artist:
    def __init__(self, name):
        self.name = name

        self.albums = []
        self.songs = []

    def add_album(self, album):
        self.albums.append(album)

    def add_song(self, song):
        self.songs.append(song)


class Playlist:
    def __init__(self, name):
        self.name = name
        self.songs = []

    def add_song(self, song):
        self.songs.append(song)

band = Artist("Bob's Awesome Band")
album = Album("Bob's First Single", band, 2013)
album.add_track("A Ballad about Cheese")
album.add_track("A Ballad about Cheese (dance remix)")
album.add_track("A Third Song to Use Up the Rest of the Space")

playlist = Playlist("My Favourite Songs")

for song in album.tracks:
    playlist.add_song(song)

In [ ]:
album.artist




In [ ]:
import datetime
from dateutil import relativedelta

DateTickInterval = 6
date1 = datetime.date(2013, 1, 1)
date2 = datetime.date(2015, 12, 31)
r = relativedelta.relativedelta(date2, date1)
NumMonths = r.years*12 + r.months
dates = [ date1 + relativedelta.relativedelta(months=n) for n in range(0,NumMonths+1,DateTickInterval)]
date_strings = [dt.strftime("%d/%m/%Y") for dt in dates]
print date_strings